In [47]:
class Persona:
    def __init__(self, nombre, edad, email):
        self.name = nombre
        self.age = edad
        self.mail = email
        self.list = []

    def saludo(self):
        print(f"Hola me llamo {self.name} y tengo {self.age} años")

    def actualizar_datos(self, nombre, email, edad):
        self.name = nombre
        self.mail = email
        self.age = edad
        print(f"Los datos han sido actualizados")

In [48]:
class Usuario(Persona):
    def __init__(self, nombre, edad, email):
        super().__init__(nombre, edad, email)
        self.reservas = []

    def promos(self, promotions):
        print("Promociones Disponibles: ")
        for promo in promotions:
            print(f"\n - {promo.description}: {promo.discount}% de descuento.\n{promo.condition}")
    
    def hacer_reserva(self, reservation, asientos):
        self.reservas.append({"Funcion": reservation, "Asientos": asientos})
        print(f"Reserva agregada para {self.name}: {reservation} con asiento {asientos}")

    def cancelar_reserva(self, reservation):
        print(f"Intentando cancelar reserva de {reservation.user.name} por {self.name}")
        print("Reservas actuales:", self.reservas)
        
        if reservation in self.reservas:
            self.reservas.remove(reservation)

        for asiento in reservation.seats:
            if asiento in reservation.function.room.asientos_ocupados:
                reservation.function.room.asientos_ocupados.remove(asiento)
            else:
                print("No se encontró reserva a cancelar")
        
        print(f"Reserva cancelada de {self.name} para la función de {reservation.function.movie.title}")


In [49]:
class Funcion:
    all_functions = []

    def __init__(self, pelicula, sala, hora):
        self.movie = pelicula
        self.room = sala
        self.horario = hora
        Funcion.all_functions.append(self)

    def __str__(self):
        return f"Pelicula: {self.movie.title} en sala {self.room.type} con horario de {self.horario}"

In [50]:
class Empleado(Persona):
    def __init__(self, nombre, edad, email, rol):
        super().__init__(nombre, edad, email)
        self.rol = rol
        self.functions = []
        self.movies = []
        self.promotions = []

    def new_function(self, function):
        if self.rol != "Limpieza":
            self.functions.append(function)
            Funcion.all_functions.append(function)
            return f"Función '{function.movie.title}' agregada con éxito!!!"
        return f"Tienes el rol {self.rol}, no puedes hacer cambios."

    def new_movie(self, pelicula):
        if self.rol != "Limpieza":
            self.movies.append(pelicula)
            return f"Película '{pelicula.title}' agregada con éxito!!!"
        return f"Tienes el rol {self.rol}, no puedes hacer cambios."

    def new_promotion(self, promocion):
        if self.rol != "Limpieza":
            self.promotions.append(promocion)
            return f"Promoción '{promocion.description}' agregada con éxito!!!"
        return f"Tienes el rol {self.rol}, no puedes hacer cambios."


In [51]:
class Espacio:
    def __init__(self, tamano):
        self.size = tamano

In [52]:
class Sala(Espacio):
    def __init__(self, tamano, tipo, capacidad):
        super().__init__(tamano)
        self.type = tipo
        self.capacity = capacidad
        self.asientos_ocupados = []

    def availability(self, asientos):
        return all(asiento not in self.asientos_ocupados for asiento in asientos)

    def occupy(self, asientos):
        if self.availability(asientos):
            if len(self.asientos_ocupados) + len(asientos) <= self.capacity:
                self.asientos_ocupados.extend(asientos)
                return True
            else:
                print("No se pueden ocupar estos asientos. Se excede la capacidad de la sala.")
                return False
        return False

In [53]:
class ZonaComida(Espacio):
    def __init__(self, tamano):
        super().__init__(tamano)
        self.products = {}

    def add_product(self,producto, precio):
        self.products[producto] = precio

In [54]:
class Pelicula:
    def __init__(self, titulo, duracion, clasificacion, genero):
        self.title = titulo
        self.duration = duracion
        self.classif = clasificacion
        self.gender = genero

    def __str__(self):
        return f"{self.title}: Duracion: {self.duration}, Clasificacion: {self.classif}, Genero: {self.gender} "

In [55]:
class Promocion:
    def __init__(self, descuento, descripcion, condiciones):
        self.description = descripcion
        self.discount = descuento
        self.condition = condiciones

    def __str__(self):
        return f"{self.description}, Descuento del {self.discount}%. Condiciones: {self.condition}"

In [56]:
class Reserva:
    def __init__(self, usuario, funcion, asientos):
        self.user = usuario
        self.seats = asientos
        self.function = funcion
        self.total = self.calcular_total()

    def calcular_total(self):
        price_2D = 85
        price_3D = 100
        price_IMAX = 120

        if self.function.room.type == "2D":
            return len(self.seats) * price_2D
        if self.function.room.type == "3D":
            return len(self.seats) * price_3D
        else:
            return len(self.seats) * price_IMAX
        
    def confirm_reserva(self):
        if self.function:
            if len(self.seats) + len(self.function.room.asientos_ocupados) > self.function.room.capacity:
                print(f"❌ No se pudo realizar la reserva. La cantidad de asientos solicitados ({len(self.seats)}) supera la capacidad de la sala ({self.function.room.capacity}).")
                return False
            elif self.function.room.occupy(self.seats):
                print(f"✅ Reserva confirmada para {self.user.name}.\nFunción: {self.function.movie.title}.\n Asientos: {self.seats}.\n Total: ${self.total}")
                return True
        else:
            print(f"❌ No se pudo realizar la reserva. Algunos asientos ya están ocupados.")
            return False

In [57]:
# ==========================
# PRUEBA COMPLETA DEL SISTEMA
# ==========================

# CREACIÓN DE OBJETOS BÁSICOS
print("\n======= CREANDO PELÍCULAS =======")
pelicula1 = Pelicula("Interstellar", 169, "PG-13", "Ciencia Ficción")
pelicula2 = Pelicula("El Padrino", 175, "R", "Drama")
print(pelicula1)
print(pelicula2)

print("\n======= CREANDO SALAS =======")
sala_2D = Sala("Grande", "2D", 100)
sala_3D = Sala("Mediana", "3D", 80)
sala_IMAX = Sala("Pequeña", "IMAX", 20)
print(f"Sala creada: {sala_2D.type}, Capacidad: {sala_2D.capacity}")
print(f"Sala creada: {sala_3D.type}, Capacidad: {sala_3D.capacity}")
print(f"Sala creada: {sala_IMAX.type}, Capacidad: {sala_IMAX.capacity}")

print("\n======= CREANDO FUNCIONES =======")
funcion1 = Funcion(pelicula1, sala_2D, "18:00")
funcion2 = Funcion(pelicula2, sala_IMAX, "20:30")
print(funcion1)
print(funcion2)

print("\n======= CREANDO EMPLEADOS =======")
empleado1 = Empleado("Carlos", 30, "carlos@email.com", "Taquillero")
empleado2 = Empleado("Luis", 25, "luis@email.com", "Limpieza")
print(f"Empleado: {empleado1.name}, Rol: {empleado1.rol}")
print(f"Empleado: {empleado2.name}, Rol: {empleado2.rol}")

print("\n======= EMPLEADOS CREANDO FUNCIONES =======")
print(empleado1.new_function(funcion1))  # Debería permitirlo
print(empleado1.new_function(funcion2))  # Debería permitirlo

print("\n======= CREANDO PROMOCIONES =======")
promo1 = Promocion(20, "Descuento de Martes", "Válido solo los martes")
promo2 = Promocion(15, "Promo de Cumpleaños", "Válido solo el día de tu cumpleaños")
print(promo1)
print(promo2)

print("\n======= EMPLEADOS CREANDO PROMOCIONES =======")
print(empleado1.new_promotion(promo1))  # Debería permitirlo
print(empleado2.new_promotion(promo2))  # NO debería permitirlo

print("\n======= CREANDO USUARIOS =======")
usuario1 = Usuario("Ana", 22, "ana@email.com")
usuario2 = Usuario("Miguel", 35, "miguel@email.com")
print(f"Usuario: {usuario1.name}, Edad: {usuario1.age}")
print(f"Usuario: {usuario2.name}, Edad: {usuario2.age}")

print("\n======= MOSTRANDO PROMOCIONES DISPONIBLES =======")
usuario1.promos([promo1, promo2])

print("\n======= USUARIOS HACIENDO RESERVAS =======")
usuario2.hacer_reserva(funcion1, 4)
reserva1 = Reserva(usuario1, funcion1, [1, 2, 3])  # Asientos 1, 2, 3 en función 1
reserva2 = Reserva(usuario2, funcion2, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])  #Compra todos los asientos
reserva3 = Reserva(usuario1, funcion2, [21]) #Compra un asiento que no existe

print("\n======= CONFIRMANDO RESERVAS =======")
reserva1.confirm_reserva()  # Debería confirmar
reserva2.confirm_reserva()  # Deberia confirmar
reserva3.confirm_reserva()  #No deberia confirmar

print("\n======= LISTA DE ASIENTOS OCUPADOS =======")
print(f"Asientos ocupados en {funcion1.movie.title}: {funcion1.room.asientos_ocupados}")
print(f"Asientos ocupados en {funcion2.movie.title}: {funcion2.room.asientos_ocupados}")

print("\n======= CANCELANDO RESERVA =======")
usuario1.cancelar_reserva(reserva1)  # Debería cancelar y liberar los asientos


print("\n======= LISTA DE ASIENTOS OCUPADOS DESPUÉS DE CANCELAR =======")
print(f"Asientos ocupados en {funcion1.movie.title}: {funcion1.room.asientos_ocupados}")
print(f"Asientos ocupados en {funcion2.movie.title}: {funcion2.room.asientos_ocupados}")

print("\n======= PROBANDO ZONA DE COMIDA =======")
zona_comida = ZonaComida("Grande")
zona_comida.add_product("Pizza 🍕", 50)
zona_comida.add_product("Refresco 🥤", 30)
print(f"Productos en zona de comida: {zona_comida.products}")



======= CREANDO PELÍCULAS =======
Interstellar: Duracion: 169, Clasificacion: PG-13, Genero: Ciencia Ficción 
El Padrino: Duracion: 175, Clasificacion: R, Genero: Drama 

======= CREANDO SALAS =======
Sala creada: 2D, Capacidad: 100
Sala creada: 3D, Capacidad: 80
Sala creada: IMAX, Capacidad: 20

======= CREANDO FUNCIONES =======
Pelicula: Interstellar en sala 2D con horario de 18:00
Pelicula: El Padrino en sala IMAX con horario de 20:30

======= CREANDO EMPLEADOS =======
Empleado: Carlos, Rol: Taquillero
Empleado: Luis, Rol: Limpieza

======= EMPLEADOS CREANDO FUNCIONES =======
Función 'Interstellar' agregada con éxito!!!
Función 'El Padrino' agregada con éxito!!!

======= CREANDO PROMOCIONES =======
Descuento de Martes, Descuento del 20%. Condiciones: Válido solo los martes
Promo de Cumpleaños, Descuento del 15%. Condiciones: Válido solo el día de tu cumpleaños

======= EMPLEADOS CREANDO PROMOCIONES =======
Promoción 'Descuento de Martes' agregada con éxito!!!
Tienes el rol Limpieza